# Preprocessing CER - Customer Behavior Trials (CBT) dataset

This dataset comes from Smart Metering Electricity Customer Behaviour Trials project and can be requested from here: https://www.ucd.ie/issda/data/commissionforenergyregulationcer/. It comes in the form of 6 zip files which needs to be unzipped and excel file specifying the stimuli each customer was subjected to or whether he was in the control group and whether he was residential or SME customer (*SME and Residential allocations.xlsx*) 

We will unzip each file and select only the residential customers from the control group. 

In [5]:
import pandas as pd
import zipfile
import numpy as np

In [7]:
path_to_data = r"path_to_your_folder_with_data"

In [8]:
df = pd.read_excel(path_to_data + '\SME and Residential allocations.xlsx')
df = df[['ID', 'Code', 'Residential - Tariff allocation', 'Residential - stimulus allocation']]
df = df[df['Code']==1]
df = df[df['Residential - Tariff allocation']=='E']
df = df[df['Residential - stimulus allocation']=='E']
ids = df['ID'].tolist()

We identified 929 client ids in the residential control group:

In [9]:
len(ids)

929

In [11]:
df_all = pd.DataFrame()
for i in range(1, 7):
    with zipfile.ZipFile(path_to_data + f'\File{i}.txt.zip', 'r') as zip_ref:
        with zip_ref.open(f'File{i}.txt') as text_file:
            df = pd.read_csv(text_file, sep=' ', header=None, names=['ID', 'datetime', 'power'])
            df = df[df['ID'].isin(ids)]
    df_all = pd.concat([df_all, df])

In [12]:
df_all.head()

,ID,datetime,power
1152,1804,19601,0.102
1153,1804,19604,0.172
1154,1804,19605,0.122
1155,1804,19602,0.175
1156,1804,19603,0.175


In [13]:
df_all.isna().sum()

ID          0
datetime    0
power       0
dtype: int64

In the dataset the datetime is encoded in the following way: 

Five digit code:  
- Day code: digits 1-3 (day 1 = 1st January 2009)
- Time code: digits 4-5 (1-48 for each 30 minutes with 1= 00:00:00 – 00:29:59)
 
We want to decode it to regular datetime format.

In [14]:
def decode_datetime(codes):
    day_codes = codes // 100
    time_codes = codes % 100
    base_date = np.datetime64('2009-01-01')
    delta_days = np.timedelta64(1, 'D') * (day_codes - 1)
    delta_minutes = np.timedelta64(30, 'm') * (time_codes - 1)
    decoded_datetimes = base_date + delta_days + delta_minutes
    return decoded_datetimes

df_all['datetime'] = decode_datetime(df_all['datetime'])

There exist duplicates due to timezone change for winter time, we drop them and resample data to hourly resolution. 

In [15]:
df_all = df_all.drop_duplicates(subset=['ID', 'datetime'])
df_all = df_all.groupby(['ID', pd.Grouper(key='datetime', freq='H')]).sum().reset_index()
df_all = df_all.rename(columns={'ID': 'id', 'power': 'consumption'})

We check whether there are any missing datetime values and add them. 

In [16]:
def count_missing_datetimes(group):
    min_datetime = group['datetime'].min()
    max_datetime = group['datetime'].max()
    all_datetimes = pd.date_range(min_datetime, max_datetime, freq='1H')
    missing = all_datetimes[~all_datetimes.isin(group['datetime'])]
    return len(missing)
    
missing_counts = df_all.groupby('id').apply(count_missing_datetimes)
print(missing_counts.sum())

4032


In [17]:
def generate_missing_datetimes(group):
    min_datetime = group['datetime'].min()
    max_datetime = group['datetime'].max()
    all_datetimes = pd.date_range(min_datetime, max_datetime, freq='1H')
    return pd.DataFrame({'id': group['id'].iloc[0], 'datetime': all_datetimes})

df_all_with_missing = df_all.groupby('id').apply(generate_missing_datetimes).reset_index(drop=True)   
df_all = df_all_with_missing.merge(df_all, how='left', on=['id', 'datetime'])           

In [18]:
df_all.to_csv('preprocessed_cbt.csv', index=False)